# 2.3 텐서 연산

- 첫 예제에서의 Dense layer를 간단히 분석해보면 다음과 같다.


- keras.layers.Dense(512, activation='relu')
    - 입력으로 2D 텐서를 받아서 출력으로 2D 텐서를 반환하는 함수
    - 더 자세히 풀어서 보면 다음과 같다.
    
    
- output = relu(dot(W, input) + b)
    - input(입력 2D 텐서), W(가중치 2D 텐서), bias(벡터)
    - 총 3가지 연산을 함
        1. input과 W의 dot product
        2. 1의 결과(2D 텐서)와 b의 덧셈
        3. ReLU 연산
    
    
- ReLU 연산
    - max(x, 0)
    - 입력이 0보다 크면 그대로 반환
    - 0보다 작으면 0을 반환
    
    
***Sequential 클래스의 add() method에 Dense 클래스가 추가될 때, Dense 객체의 build() 메서드가 호출되며 가중치(kernel) W와 편향(bias) b가 생성되고, Dense 객체의 kernel과 bias 인스턴스 변수에 저장됨***

## 2.3.1 원소별 연산 (element-wise operation)

- relu()와 덧셈은 원소별 연산
- 원소별 연산이란 텐서내의 원소끼리 이루어지는 연산을 뜻함
- 넘파이에 원소별 연산이 이미 구현되어 있고 훨씬 빠름
- 주로 * 연산자를 이용

In [1]:
import numpy as np

# 직접 구현한 relu()
def naive_relu(x):
    assert len(x.shape) == 2
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
            
    return x


# 직접 구현한 add()
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
            
    return x


a = np.random.randint(-10, 10, (3, 3))
b = np.random.randint(-10, 10, (3, 3))

print("a")
print(a)
print("b")
print(b)
print("")

print("relu(a)")
print(naive_relu(a))
print("")

print("add(a, b)")
print(naive_add(a, b))
print("")


# numpy의 구현된 maximum(relu()와 동일한 기능)
print("numpy의 relu")
print(np.maximum(a, 0.))
print("")

# numpy의 구현된 add()
print("numpy의 add")
print(a + b)

a
[[-1 -4 -3]
 [-7 -3  6]
 [ 3 -5 -4]]
b
[[-9 -2 -2]
 [-8 -5 -3]
 [-8  3 -5]]

relu(a)
[[0 0 0]
 [0 0 6]
 [3 0 0]]

add(a, b)
[[-10  -6  -5]
 [-15  -8   3]
 [ -5  -2  -9]]

numpy의 relu
[[0. 0. 0.]
 [0. 0. 6.]
 [3. 0. 0.]]

numpy의 add
[[-10  -6  -5]
 [-15  -8   3]
 [ -5  -2  -9]]


## 2.3.2 브로드캐스팅 (broadcasting)
- 작은 텐서가 큰 텐서의 크기에 맞춰지는 것을 브로드캐스팅이라 함
- 브로드캐스팅의 2단계
    1. 큰 텐서의 ndim에 맞도록 작은 텐서에 축(브로드캐스팅 축이라 함)이 추가됨
    2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복됨

In [2]:
# 직접 구현한 broadcasting
# 실제로는 알고리즘 수준에서 더 빠르게 구현되어 있을 것임, 간단히 구현만 해본 것
import numpy as np

def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
            
    return x


# 마지막 차원 맞춰주기 (5를 같게함)
a = np.random.randint(-10, 10, [10, 5])
b = np.random.randint(-10, 10, [5])
c = np.random.randint(10, size=[64, 3, 32, 10])
d = np.random.randint(10, size=[32, 10])

print("a :", a.shape)
print("b :", b.shape)
print("")

print("a+b (broadcasting) :", (a+b).shape)
print("naive_add_matrix_and_vector(a, b) :", naive_add_matrix_and_vector(a, b).shape)
print("")

# 이렇게도 broadcasting이 일어남
print("c :", c.shape)
print("d :", d.shape)
print("c+d (broadcasting) :", (c+d).shape)
print("")

a : (10, 5)
b : (5,)

a+b (broadcasting) : (10, 5)
naive_add_matrix_and_vector(a, b) : (10, 5)

c : (64, 3, 32, 10)
d : (32, 10)
c+d (broadcasting) : (64, 3, 32, 10)



## 2.3.3 텐서 점곱 (dot product)
- 원소별 연산과 달리 입력 텐서의 원소들을 결합시킴
- 넘파이, 케라스는 dot연산자를 사용(텐서플로에서는 다름)
- 일반적인 행렬 곱셈 연산이랑 같음
- A dot product B에서, (A의 행길이 == B의 열길이)를 만족해야 함

In [3]:
# 파이썬으로 점곱 구현
import numpy as np

# dot(vector, vector)
def naive_vv_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z = 0.
    for i in range(x.shape[0]):
        z += x[i]*y[i]
        
    return z


# dot(matrix, vector)
def naive_mv_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(y.shape[0]):
            z[j] += x[i][j]*y[j]
            
    return z


# dot(matrix, vector)
def naive_mv_dot2(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i][:], y)
        
    return z


# dot(matrix, matrix)
def naive_mm_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x_row = x[i,:]
            y_col = y[:,j]
            z[i, j] = naive_vv_dot(x_row, y_col)
            
    return z
    

a = np.array([[1, 2],[3, 4]])
b = np.array([[3, 4],[5, 6]])

print(naive_mm_dot(a, b))
print(np.dot(a, b))

[[13. 16.]
 [29. 36.]]
[[13 16]
 [29 36]]


## 2.3.4 텐서 크기 변환 (tensor reshaping)
- 특정 크기에 맞게 열과 행을 재배열 하는 것
- 주로 전치(transposition)를 사용함
- reshape()를 사용

In [4]:
import numpy as np

x = np.array([[0., 1.], [2., 3.], [4., 5.]])
print(x)
print("")

# reshape를 통해 형태 transposition이랑 같게 해도 원소 배치는 다름
# reshape는 형태만 재배열하는것
y = x.reshape((x.shape[1], x.shape[0]))
# transposition은 x[:, i] => x[i, :]
z = np.transpose(x)

print(y)
print("")

print(z)
print("")

[[0. 1.]
 [2. 3.]
 [4. 5.]]

[[0. 1. 2.]
 [3. 4. 5.]]

[[0. 2. 4.]
 [1. 3. 5.]]



## 2.3.5 텐서 연산의 기하학적 해석
- 텐서 연산이 조작하는 텐서의 내용은 어떤 기하학적 공간에 있는 좌표 포인트로 해석할 수 있음
- 따라서 모든 텐서 연산은 기하학적 해석이 가능함

In [5]:
import numpy as np

# Ex1
# 벡터의 덧셈은 기히학적으로 화살표 벡터의 연결이라 할 수 있음
A = np.array([0.5, 1])
B = np.array([1, 0.25])
print(A+B)
print("")

# Ex2
# 행렬 [[cos(theta), sin(theta)],[-sin(theta), cos(theta)]]의 dot product는 
# theta 각도로 2D 벡터를 회전하는 것이라 할 수 있음

v = np.array([1., 1.])
theta = np.pi / 2.
R = np.array([[np.cos(theta), np.sin(theta)],[-np.sin(theta), np.cos(theta)]])

print(v)
print(np.dot(v, R))

[1.5  1.25]

[1. 1.]
[-1.  1.]


## 2.3.6 딥러닝의 기하학적 해석
신경망은 전체적으로 텐서 연산의 연결들로 구성되고 모든 텐서 연산은 입력 데이터의 기하학적 변환이다. 따라서 단순한 단계들이 길게 이어저 구현된 신경망을 고차원 공간에서 매우 복잡한 기하학적 변환을 하는 것으로 해석할 수 있다.

3D로 비유하자면, 하나는 빨강, 하나는 파랑인 2개의 색종이가 있다고 가정하고 두 장을 겹친 후 뭉쳐서 작은 공으로 만든다면, 이 종이 공이 입력 데이터, 색종이는 분류 문제의 데이터 클래스라고 할 수 있다.
여기서 신경망이 해야 할 일은 종이 공을 펼쳐서 두 클래스가 다시 깔끔하게 분리되는 변환을 찾는 것이다.

**종이 공을 하나하나 펼치며 복잡한 분해 과정을 처리하는 것 = 심층 신경망의 연결된 각 층에서의 간단한 변환으로 복잡한 데이터를 풀어주는 것**